## implementing word embedding (glove) and CNN classifier on genie data

In [1]:
GLOVE_DIR='./data/'
PATH ='../GenieData/'

In [2]:
MAX_NB_WORDS=20000
MAX_SEQUENCE_LENGTH=1000
VALIDATION_SPLIT=0.2
EMBEDDING_DIM=100

In [3]:
import os
import pandas as pd
import numpy as np
import string

2. Preparing Data

In [4]:
df=pd.read_csv(PATH+'allqns.csv',header=None)

In [5]:
df.head()

,0,1,2,3
0,wallet,manage_payment_method,pay sameer,wallet.manage_payment_method
1,wallet,manage_payment_method,pay sameer Rs 1000,wallet.manage_payment_method
2,wallet,manage_payment_method,cancel payment,wallet.manage_payment_method
3,wallet,manage_payment_method,add Rs 1000 to my Wallet,wallet.manage_payment_method
4,wallet,manage_payment_method,add Rs 1000 to my purse,wallet.manage_payment_method


In [6]:
df=df.drop(labels=[0,1],axis=1)

In [7]:
df.dropna(inplace=True)

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.columns=['qns','category']

In [10]:
df['qns']=df['qns'].apply(lambda x:x.lower())

In [11]:
questions=df['qns'].tolist()

Sequencing

In [12]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

/home/gt5/miniconda3/envs/pnltk/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [13]:
tokenizer=Tokenizer()

In [14]:
tokenizer.fit_on_texts(questions)

In [15]:
sequences=tokenizer.texts_to_sequences(questions)

In [16]:
wordindex=tokenizer.word_index

In [17]:
len(wordindex)

776

In [18]:
lens=[len(x) for x in sequences]

In [19]:
sequences=pad_sequences(sequences)

In [20]:
sequences.shape

(3444, 21)

category mapping

In [21]:
categories=list(set(df['category'].tolist()))

In [22]:
categories

['lounge.book',
 'flight.cancel',
 'calendar.delete',
 'alarm.delete',
 'flight.check-in',
 'order_food.cancel',
 'transactions.export_details',
 'email.send',
 'alarm.reschedule',
 'content.read',
 'weather.read',
 'calendar.read',
 'hotel.cancel',
 'self-drive.book',
 'map.direction',
 'flight.book',
 'flight.#',
 'content.#',
 'profile.show',
 'sms.send',
 'tip_genie.tip',
 'table_booking.book',
 'refer_and_earn.make_referrel',
 'calendar.schedule',
 'movies.book',
 'call.place_call',
 'lounge.cancel',
 'alarm.set',
 'wallet.manage_payment_method',
 'lounge.#',
 'events.book',
 'calculator.calculate',
 'flight.status',
 'map.nearby',
 'porter.book',
 'calendar.modify',
 'table_booking.cancel',
 'flight.upgrade',
 'flight.modify',
 'hotel.modify',
 'movies.cancel',
 'hotel.book',
 'order_food.book',
 'map.#']

In [23]:
categories_index={key:categories.index(value) for (key,value) in enumerate(df['category'].tolist())}

In [24]:
categories_index

{0: 28,
 1: 28,
 2: 28,
 3: 28,
 4: 28,
 5: 28,
 6: 28,
 7: 28,
 8: 28,
 9: 28,
 10: 28,
 11: 28,
 12: 28,
 13: 28,
 14: 28,
 15: 28,
 16: 28,
 17: 28,
 18: 28,
 19: 28,
 20: 28,
 21: 28,
 22: 41,
 23: 41,
 24: 41,
 25: 41,
 26: 41,
 27: 41,
 28: 41,
 29: 41,
 30: 41,
 31: 41,
 32: 41,
 33: 41,
 34: 41,
 35: 41,
 36: 41,
 37: 41,
 38: 41,
 39: 41,
 40: 41,
 41: 41,
 42: 41,
 43: 41,
 44: 41,
 45: 41,
 46: 41,
 47: 41,
 48: 41,
 49: 41,
 50: 41,
 51: 41,
 52: 41,
 53: 41,
 54: 41,
 55: 41,
 56: 41,
 57: 41,
 58: 41,
 59: 41,
 60: 41,
 61: 41,
 62: 41,
 63: 41,
 64: 41,
 65: 41,
 66: 41,
 67: 41,
 68: 41,
 69: 41,
 70: 41,
 71: 41,
 72: 41,
 73: 41,
 74: 41,
 75: 41,
 76: 41,
 77: 41,
 78: 41,
 79: 41,
 80: 41,
 81: 41,
 82: 39,
 83: 12,
 84: 12,
 85: 41,
 86: 41,
 87: 41,
 88: 41,
 89: 41,
 90: 25,
 91: 25,
 92: 25,
 93: 25,
 94: 25,
 95: 25,
 96: 25,
 97: 25,
 98: 25,
 99: 25,
 100: 25,
 101: 25,
 102: 25,
 103: 25,
 104: 25,
 105: 25,
 106: 25,
 107: 31,
 108: 31,
 109: 31,
 110: 31,


In [25]:
values=list(categories_index.values())

In [26]:
from keras.utils import to_categorical
import numpy as np
labels=to_categorical(np.asarray(values))

In [27]:
labels.shape

(3444, 44)

In [28]:
sequences.shape

(3444, 21)

randomfying

In [29]:
indices=np.arange(sequences.shape[0])

In [30]:
np.random.shuffle(indices)

In [31]:
sequences=sequences[indices]

In [32]:
labels=labels[indices]

In [33]:
nb_validation_samples=int(VALIDATION_SPLIT * sequences.shape[0])

In [34]:
nb_validation_samples

688

In [35]:
x_train=sequences[:-nb_validation_samples]
x_test=sequences[-nb_validation_samples:]
y_train=labels[:-nb_validation_samples]
y_test=labels[-nb_validation_samples:]

preparing embedded layer

In [36]:
embedding_index={}

In [37]:
f=open(os.path.join(GLOVE_DIR,'glove.6B.100d.txt'))

In [38]:
for line in f:
    values=line.split()
    word=values[0]
    embeddings=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=embeddings
f.close()

In [39]:
print(len(embedding_index))

400000


In [40]:
embedding_matrix=np.zeros((len(wordindex)+1,EMBEDDING_DIM))

In [41]:
for word,i in wordindex.items():
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

embedding layer

In [42]:
from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, Flatten, Dense, GlobalMaxPooling1D
#Embedding?

In [43]:
from keras.layers import Embedding
from keras.models import Model
embedding_layer=Embedding(len(wordindex)+1,EMBEDDING_DIM,weights=[embedding_matrix],trainable=False)

convnet

In [44]:
sequence_input=Input(shape=(21,),dtype='int32')

In [45]:
embedded_sequences=embedding_layer(sequence_input)

In [46]:
x=Conv1D(128,5,activation='relu')(embedded_sequences)

In [47]:
x=Conv1D(128,5,activation='relu')(x)

In [48]:
x=MaxPooling1D(2)(x)

In [49]:
x=Conv1D(128,5,activation='relu')(x)

In [50]:
x=GlobalMaxPooling1D()(x)

In [51]:
x=Dense(128, activation='relu')(x)

In [53]:
preds=Dense(len(labels),activation='softmax')(x)

In [54]:
model=Model(sequence_input,preds)

In [55]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 21, 100)           77700     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 17, 128)           64128     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 128)           82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 6, 128)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 2, 128)            82048     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
__________

In [58]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=128)

ValueError: Error when checking target: expected dense_2 to have shape (3444,) but got array with shape (44,)